# ESPIDAM SIR Age-Structured Example With Vaccination

*Authors: Sebastiaan Weytjens, Pieter Libin, Niel Hens*

### Age-Structured SIR: Adding Vaccination

To enable vaccination in the SIR model, we add two compartments: SV and IV, vaccinated susceptible and infected individuals, respectively. We modify the ODE equations as follows, where $\rho_i$ indicates the vaccination rate for age class i, $VE_S$ the susceptibility of vaccinated individuals, and $VE_I$ the infectiousness of vaccinated infected individuals:

$
\begin{aligned}
\frac{{dS_i}}{{dt}} &= - \rho_i \cdot S_i -\beta \cdot S_i \cdot \sum_j C_{i j} \cdot (I_j + VE_I \cdot IV_j) / N_j \\
\frac{{dSV_i}}{{dt}} &= \rho_i \cdot S_i -\beta \cdot VE_S \cdot SV_i \cdot \sum_j C_{i j} \cdot (I_j + VE_I \cdot IV_j) / N_j \\
\frac{{dI_i}}{{dt}} &= - \rho_i \cdot I_i - \gamma \cdot I_i + \beta \cdot S_i \cdot \sum_j C_{i j} \cdot (I_j + VE_I \cdot IV_j) / N_j \\
\frac{{dIV_i}}{{dt}} &= \rho_i \cdot I_i - \gamma \cdot IV_i + \beta \cdot VE_S \cdot SV_i \cdot \sum_j C_{i j} \cdot (I_j + VE_I \cdot IV_j) / N_j \\
\frac{{dR_i}}{{dt}} &= \gamma \cdot I_i + \gamma \cdot IV_i
\end{aligned}
$

#### Dependencies

In [ ]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from sir_helpers import *
import math
from numpy.linalg import eig

#### Model Description

We will start with the model description. Again, we give you a head start by filling in the compartments, the contact matrix and the initial compartment values. Complete the force of infection and the required rates.

In [ ]:
# Compartments
compartments = {
    "S_c": [],
    "SV_c": [],
    "I_c": [],
    "IV_c": [],
    "R_c": [],
    "S_a": [],
    "SV_a": [],
    "I_a": [],
    "IV_a": [],
    "R_a": []
}

# Contact Matrix
def contacts(i, j):
    contact_matrix = [[18, 9], [3, 12]]
    return contact_matrix[i][j]

#Transition rates
#Ns are populations sizes of all age classes ac_i is the age class i (as shown in the aforementioned formulas), acs is the number of age classes
def foi(i, iv, params, Ns, ac_i, acs):
    foi_value = 0

    #TODO: FOI calculation
        
    return foi_value

def i_r(params):
    #TODO: I to R rate
    pass

def i_iv(params, ac_i):
    #TODO: vaccination rate, which depends on the age class
    pass

def s_sv(params, ac_i):
    #TODO: vaccination rate, which depends on the age class
    pass

def initialise_modelstate(modelstate, seeds, N_c, N_a):
    modelstate["S_c"] = [N_c - seeds]
    modelstate["SV_c"] = [0]
    modelstate["I_c"] = [seeds]
    modelstate["IV_c"] = [0]
    modelstate["R_c"] = [0]

    modelstate["S_a"] = [N_a - seeds]
    modelstate["SV_a"] = [0]
    modelstate["I_a"] = [seeds]
    modelstate["IV_a"] = [0]
    modelstate["R_a"] = [0]

    return modelstate

#### Solvers for the model

Next, we will use ODEs to simulate our model. Complete the ode_system function with the correct calculations.

In [ ]:
# ODE Solver

def ode_system(y0, t, parameters):
    params = parameters["disease_params"]
    Ns = parameters["Ns"]

    s_c, sv_c, i_c, iv_c, r_c, s_a, sv_a, i_a, iv_a, r_a = y0
    ds_c, dsv_c, di_c, div_c, dr_c, ds_a, dsv_a, di_a, div_a, dr_a = 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

    #TODO: Calculate the new individuals for each compartment for children and adults

    return ds_c, dsv_c, di_c, div_c, dr_c, ds_a, dsv_a, di_a, div_a, dr_a    

def ode_solver(model_state, end_t, params, Ns):
    all_parameters = {
        "disease_params": params,
        "Ns": Ns
    }

    # Initial conditions (modelstates, timesteps)   
    y0 = (model_state["S_c"][0], model_state["SV_c"][0], model_state["I_c"][0], model_state["IV_c"][0], model_state["R_c"][0], model_state["S_a"][0], model_state["SV_a"][0], model_state["I_a"][0], model_state["IV_a"][0], model_state["R_a"][0])
    t = np.linspace(0, end_t, end_t)
    
    # Solving the ODE system
    ret = odeint(ode_system, y0, t, args=(all_parameters,))
    s_c, sv_c, i_c, iv_c, r_c, s_a, sv_a, i_a, iv_a, r_a = ret.T

    new_model_state = {
        "S_c": list(s_c),
        "SV_c": list(sv_c),
        "I_c": list(i_c),
        "IV_c": list(iv_c),
        "R_c": list(r_c),
        "S_a": list(s_a),
        "SV_a": list(sv_a),
        "I_a": list(i_a),
        "IV_a": list(iv_a),
        "R_a": list(r_a)
    }

    return new_model_state


#### Model Parameters

If you implemented everything correctly, you can use the parameters below to simulate your age-structured SIR model with vaccinations.

In [ ]:
N = 1000000
N_c = 0.25 * N #total population of children
N_a = 0.75 * N #total population of adults
seeds = 1 #initial number of infected individuals

gamma = 1/7 #recovery rate
R0 = 3 #basic reproduction number
f = [0.25, 0.75] # fraction of children and adults
m = np.array([[18*f[0]/f[0], 9*f[0]/f[1]], [3*f[1]/f[0], 12*f[1]/f[1]]]) #normalised contact matrix
spectral_radius = max(eig(m)[0]) #spectral radius of the contact matrix

params = {
    "beta": R0 * gamma / spectral_radius, #transmission rate
    "gamma": gamma, #recovery rate
    "ve_s": 0.4, #susceptibility of vaccinated individuals
    "ve_i": 0.5, #infectivity of vaccinated infected individuals
    "rho_c": 0.01, #daily vaccination rate of children
    "rho_a": 0.01 #daily vaccination rate of adults
}

end_t = 150 #days to simulate

#### Vaccination Scenarios

Below you can run a scenario with vaccination for adults and children as defined above in the parameters code section. We plot the infected and vaccinated infected individuals over time, both for children and adults.

In [ ]:
modelstate = initialise_modelstate(compartments, seeds, N_c, N_a)
results = ode_solver(modelstate, end_t, params, [N_c, N_a])

plot_ODE_w_vacc(results, "Children and adults vaccinated", 200000)

##### Decreased infectivity for vaccinated individuals

Let us assume that vaccinated infected individuals are less infectious than defined in the previous section. Adjust the parameters, perform a simulation and interpret the results as we did in the previous scenario. How did this affect the disease outcome?

In [ ]:
#TODO: Run a scenario with decreased infectivity of vaccinated individuals and analyse the results.

#### Vaccine Policies

Reset the parameters to the initial configuration. Now, let's compare what happens when we vaccinate children vs adults, what are the differences in attack rate?

##### Only adults vaccinated

In [ ]:
#TODO: Run a scenario where we only vaccinate adults and analyse the results.


##### Only children vaccinated

In [ ]:
#TODO: Run a scenario where we only vaccinate children and analyse the results.

#### *Extra*: Try to implement a stochastic version of the prior exercises. 